In [37]:
import pandas as pd
import numpy as np

In [38]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv

--2023-09-26 00:41:07--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1423529 (1.4M) [text/plain]
Saving to: ‘housing.csv.2’

housing.csv.2       100%[===================>]   1.36M  1.23MB/s    in 1.1s    

2023-09-26 00:41:09 (1.23 MB/s) - ‘housing.csv.2’ saved [1423529/1423529]



In [39]:
df = pd.read_csv('housing.csv')
filtered_data = df[df['ocean_proximity'].isin ( ['<1H OCEAN' ,'INLAND']) ] 
missing_values = filtered_data.isnull().sum()

column_with_missing_values = missing_values[missing_values > 0].index[0]

print(column_with_missing_values)

total_bedrooms


In [40]:
median_population = filtered_data['population'].median()
print(median_population)

1195.0


In [41]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

selected_columns = [
    'latitude', 'longitude', 'housing_median_age', 'total_rooms', 
    'total_bedrooms', 'population', 'households', 'median_income', 
    'median_house_value'
]
filtered_data = filtered_data[selected_columns]

filtered_data = filtered_data.sample(frac=1, random_state=42)

train_data, val_data, test_data = np.split(filtered_data, [int(0.6*len(filtered_data)), int(0.8*len(filtered_data))])

train_data['median_house_value'] = np.log1p(train_data['median_house_value'])
val_data['median_house_value'] = np.log1p(val_data['median_house_value'])

train_data_fill_zero = train_data.fillna(0)
val_data_fill_zero = val_data.fillna(0)

mean_total_bedrooms = train_data['total_bedrooms'].mean()
train_data_fill_mean = train_data.fillna({'total_bedrooms': mean_total_bedrooms})
val_data_fill_mean = val_data.fillna({'total_bedrooms': mean_total_bedrooms})

def train_and_evaluate(data_train, data_val):
    X_train = data_train.drop('median_house_value', axis=1)
    y_train = data_train['median_house_value']
    X_val = data_val.drop('median_house_value', axis=1)
    y_val = data_val['median_house_value']
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    return rmse

rmse_fill_zero = train_and_evaluate(train_data_fill_zero, val_data_fill_zero)
rmse_fill_mean = train_and_evaluate(train_data_fill_mean, val_data_fill_mean)

print("RMSE with 0:", round(rmse_fill_zero, 2))
print("RMSE with mean:", round(rmse_fill_mean, 2))

if round(rmse_fill_zero, 2) < round(rmse_fill_mean, 2):
    print("Better option: With 0")
elif round(rmse_fill_zero, 2) > round(rmse_fill_mean, 2):
    print("Better option: With mean")
else:
    print("Both options are equally good")


RMSE with 0: 0.34
RMSE with mean: 0.34
Both options are equally good


/home/peng/ML_Zoom_Camp/ML_Venv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [20]:
from sklearn.linear_model import Ridge

# Function to train and evaluate a regularized linear regression model with given alpha
def train_and_evaluate_ridge(data_train, data_val, alpha):
    X_train = data_train.drop('median_house_value', axis=1)
    y_train = data_train['median_house_value']
    X_val = data_val.drop('median_house_value', axis=1)
    y_val = data_val['median_house_value']
    
    model = Ridge(alpha=alpha)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    return rmse

alphas = [0, 0.000001, 0.0001, 0.001]
best_alpha = None
best_rmse = float('inf')

for alpha in alphas:
    rmse = train_and_evaluate_ridge(train_data_fill_zero, val_data_fill_zero, alpha)
    print(f"Alpha = {alpha}: RMSE = {round(rmse, 9)}")
    print(f"Rounded Alpha = {alpha}: RMSE = {round(rmse, 2)}")
    if rmse < best_rmse:
        best_alpha = alpha
        best_rmse = rmse

print("Best alpha:", best_alpha)


Alpha = 0: RMSE = 0.337022325
Rounded Alpha = 0: RMSE = 0.34
Alpha = 1e-06: RMSE = 0.337022325
Rounded Alpha = 1e-06: RMSE = 0.34
Alpha = 0.0001: RMSE = 0.337022325
Rounded Alpha = 0.0001: RMSE = 0.34
Alpha = 0.001: RMSE = 0.337022322
Rounded Alpha = 0.001: RMSE = 0.34
Best alpha: 0.001


Standard Deviation of RMSE Scores: 1483.007
